In [1]:
import os
import re
import pymupdf
import django_setup
from IPython.display import Markdown, display
from bs4 import BeautifulSoup as bs

from django.conf import settings

gemini_api_key = settings.GEMINI_API_KEY

In [2]:
from sapl.compilacao.models import Dispositivo

In [3]:
count_words = 0
chunks = []
for d in Dispositivo.objects.filter(
    ta_id=2222,
    dispositivo_subsequente__isnull=True,
    tipo_dispositivo_id__gte=119
    ):
    if not d.nivel:
        continue
    relatives_chunks = d.render_relative_chunk(nivel=119)
    if relatives_chunks:
        chunks.append((d, bs(relatives_chunks, 'html.parser').get_text()))
        count_words += len(chunks[-1][-1].split())
    #print(d.id, bs(f'{d.rotulo} {d.texto}', 'html.parser').get_text()) #.prettify())

WARNING 2025-01-16 19:59:58,256 py.warnings:_showwarnmsg:109 /tmp/ipykernel_25896/2559035337.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  chunks.append((d, bs(relatives_chunks, 'html.parser').get_text()))



In [4]:
count_words

46924

In [5]:
for c in chunks[0:10]:
    print(c)
    print('-' * 30)

(<Dispositivo: Art. 1º.>, 'Art. 1º. - A Câmara é o órgão Legislativo do Município e tem sede própria denominada Edifício Eufrásio Pereira Rocha, localizado no "PALÁCIO DAS ABELHAS", situado na Praça da Bandeira, 96 - Centro.\n§ 1º - Na sede da Câmara não se realizarão atos estranhos à sua função, salvo por deliberação do Plenário ou concessão da Mesa Diretora.\n§ 2º - Havendo motivo relevante, ou de força maior, a Câmara Municipal poderá, por deliberação da Mesa, ad referendum da maioria absoluta dos Vereadores, reunir-se em outro local.\n§ 3º - Nenhum objeto pertencente ao Poder Legislativo Municipal poderá ser retirado de seu recinto a título de empréstimo, salvo por deliberação plenária.')
------------------------------
(<Dispositivo: § 1º>, 'Art. 1º. - A Câmara é o órgão Legislativo do Município e tem sede própria denominada Edifício Eufrásio Pereira Rocha, localizado no "PALÁCIO DAS ABELHAS", situado na Praça da Bandeira, 96 - Centro.\n§ 1º - Na sede da Câmara não se realizarão at